In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pyproj
import geopandas as gpd
from shapely.geometry import Point

## 新增設施

In [3]:
facility_data = pd.read_csv("landmark.csv")
test = pd.read_csv("30_Public Dataset_Public Sumission Template_v2/public_dataset.csv")
test.head(5)

,ID,縣市,鄉鎮市區,路名,土地面積,使用分區,移轉層次,總樓層數,主要用途,主要建材,...,屋齡,建物面積,車位面積,車位個數,橫坐標,縱坐標,備註,主建物面積,陽台面積,附屬建物面積
0,PU-1,新北市,樹林區,大有路,2.039378,None,3,7,住家用,鋼筋混凝土造,...,13.916667,0.472202,0.146176,1.0,288745,2760266,NaN,0.849664,0.146161,-0.438452
1,PU-2,新北市,三重區,力行路一段,-0.077786,None,5,5,住家用,鋼筋混凝土造,...,40.833333,-0.546455,-0.819326,0.0,298751,2773492,NaN,-0.004102,0.064257,-0.438452
2,PU-3,新北市,板橋區,民生路三段,-0.555766,None,2,20,集合住宅,鋼筋混凝土造,...,1.333333,0.124032,-0.167419,1.0,297000,2769106,NaN,-0.285862,-0.775260,1.323042
3,PU-4,新北市,泰山區,明志路二段,-0.744838,None,6,16,住家用,鋼筋混凝土造,...,27.500000,-0.027072,0.115280,1.0,293076,2771160,NaN,0.027582,0.316794,-0.139532
4,PU-5,台北市,信義區,忠孝東路五段,-0.610923,None,3,7,住家用,鋼筋混凝土造,...,40.500000,-0.863600,-0.819326,0.0,308103,2770220,NaN,-0.883328,-0.560262,-0.075477


In [ ]:
def train_coordination(x):
  x1, y1 = x['橫坐標'] ,x['縱坐標']
  proj = pyproj.Transformer.from_crs(3826, 4326, always_xy=True)
  x2, y2 = proj.transform(x1, y1)
  return x2, y2

train[['lon','lat']] = train.apply(train_coordination,axis=1,result_type='expand')

In [ ]:
# 轉換 new_train 和 facility_data 為 GeoDataFrame
test['geometry'] = [Point(xy) for xy in zip(test['lon'], test['lat'])]
test = gpd.GeoDataFrame(new_train, geometry='geometry')

# 設定crs座標 TWD97
facility_data = gpd.GeoDataFrame(facility_data, geometry=gpd.points_from_xy(facility_data['經度'], facility_data['緯度']))
facility_data.crs = "EPSG:3826"
new_train.crs = "EPSG:3826"